In [15]:
import pandas as pd

df1 = pd.read_csv("./posts1352.csv", encoding_errors= 'replace')

In [13]:
stopword_extra = ["always","want","even","still","ever","also","already","yet","basically","actually","need","please","ago","probable","probably","however","instead","quite","nt","na","u","gon","lol","im","ca","us","cnt","wo","em","etc","ll","aint","r","cant","shouldnt","wont","lah","dont","never"]

In [16]:
from nltk.tokenize import sent_tokenize

In [17]:
df2 = df1.dropna()

#df1.fillna("Zero")

In [18]:
df2["Post"] = df2["Post"].apply(lambda x: sent_tokenize(x)) #this only "tokenize" post with multuple lines into sentences

C:\Users\qing5\AppData\Local\Temp\ipykernel_12968\3973988239.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Post"] = df2["Post"].apply(lambda x: sent_tokenize(x)) #this only "tokenize" post with multuple lines into sentences


In [66]:
#df1

In [19]:
df2 = df2.explode('Post')

df2 = df2.reset_index(drop=True)

In [20]:
df2

,No,Category,SubCategory,Post,HateSpeech
0,1,Race,Chinese,Chinese are pigs!,1
1,2,Race,Indian,I don't like pajeets they smell like curry,1
2,4,Race,Indian,"Indians always smell stinky and weird, what is...",1
3,5,Gender,Female,Women are so emotional they can't handle anyth...,1
4,6,Race,Malay,Malays are dependant on subsidiaries from the ...,1
...,...,...,...,...,...
1633,1349,Religion,Buddhism,You cannot build a temple in an area with majo...,1
1634,1350,Religion,Buddhism,"Muslims should not enter temple, or they might...",1
1635,1351,Gender,Male,Men that don't have job are useless piece of s...,1
1636,1352,Gender,Female,Women are dick machines.,1


In [6]:
#df2

,No,Category,SubCategory1,Post,HateSpeech
0,1,Race,Chinese,Chinese are pigs!,1
1,2,Race,Indian,I don't like pajeets they smell like curry,1
3,4,Race,Indian,"Indians always smell stinky and weird, what is...",1
4,5,Gender,Female,Women are so emotional they can't handle anyth...,1
5,6,Race,Malay,Malays are dependant on subsidiaries from the ...,1
...,...,...,...,...,...
1971,1349,Religion,Buddhism,You cannot build a temple in an area with majo...,1
1972,1350,Religion,Buddhism,"Muslims should not enter temple, or they might...",1
1973,1351,Gender,Male,Men that don't have job are useless piece of s...,1
1974,1352,Gender,Female,Women are dick machines.,1


## Skip here

In [ ]:
df1['HateSpeech'].value_counts(normalize=True, dropna=False)

In [ ]:
df1['Category'].value_counts(normalize=True,dropna=False)

In [ ]:
religion = df1[df1['Category']=='Religion']

In [ ]:
religion['SubCategory1'].value_counts(normalize=True)

In [ ]:
religion['HateSpeech'].value_counts()

In [ ]:
race= df1[df1['Category']=='Race']

In [ ]:
race['SubCategory1'].value_counts()

In [ ]:
race['HateSpeech'].value_counts()

In [ ]:
gender = df1[df1['Category']=='Gender']

In [ ]:
gender['SubCategory1'].value_counts(normalize=True)

In [ ]:
gender['HateSpeech'].value_counts()

In [ ]:
df2['HateSpeech'].value_counts(dropna=False)

## Start at here 

In [21]:
import string
import emoji
import re
import nltk
from textblob import TextBlob
from nltk.corpus import wordnet
from nltk.metrics import edit_distance
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

import spacy
nlp = spacy.load('en_core_web_sm') #, disable=['parser', 'ner']) 

stopwords = nltk.corpus.stopwords.words('english')
stopwords += stopword_extra

negative_list = ['not','never','ain','aint','no','neither','nor','nt','cant','dont',"cnt",'wont',"shouldnt"]

def preprocess(text):
    
    #1. Generating the list of words in the tweet (hastags and other punctuations removed)
    # .correct() is added to automatically correct typos and grammar errors in datasets
    text_blob = TextBlob(text)
    text = ' '.join(text_blob.words)
    
    # remove number
    text = re.sub(r'[0-9]', '', text)
    
    # lowercase
    text = text.lower()
    
    text = text.replace('/',' ')
    
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
        
    
    text = word_tokenize(text)
    
    #keep tokens that are alphabet characters
    text = [t for t in text if t.isalpha()]
    
    # replace the negation token
    replacer  = AntonymReplacer()
    text = replacer.replace_negations(text)
    
    # remove the stopwords
    text = [i for i in text if i not in stopwords]
       
    # lemmatize the text
    #text = lemmatization(text)
    
    #remove empty token
    text = [t for t in text if len(t) > 0]
    
    return text


def lemmatization(sent, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    doc = nlp(" ".join(sent)) 
    texts_out = [token.lemma_ for token in doc if token.pos_ in allowed_postags]
    return texts_out

class AntonymReplacer(object):
    def replace(self, word, pos=None):
        antonyms = set()

        for syn in wordnet.synsets(word, pos=pos):
            for lemma in syn.lemmas():
                for antonym in lemma.antonyms():
                    antonyms.add(antonym.name())

        if len(antonyms) == 1:
            return antonyms.pop()
        else:
            return None

    def replace_negations(self, sent):
        i, l = 0, len(sent)
        words = []

        while i < l:
            word = sent[i]

            if word in negative_list and i+1 < l:
                ant = self.replace(sent[i+1])

                if ant:
                    words.append(ant)
                    i += 2
                    continue

            words.append(word)
            i += 1

        return words

In [22]:
#Preprocess the input data
cleaned_text = []
counter = 1

for text in df2["Post"]:
    if counter % 100 == 0:
        print(f'Preprocessing row {counter}')
    counter+=1
    
    cleaned_text.append(preprocess(text))

Preprocessing row 100
Preprocessing row 200
Preprocessing row 300
Preprocessing row 400
Preprocessing row 500
Preprocessing row 600
Preprocessing row 700
Preprocessing row 800
Preprocessing row 900
Preprocessing row 1000
Preprocessing row 1100
Preprocessing row 1200
Preprocessing row 1300
Preprocessing row 1400
Preprocessing row 1500
Preprocessing row 1600


In [23]:
df2["PreprocessedToken"] = cleaned_text

In [24]:
for i in range(len(df2)):

    if len(df2["PreprocessedToken"][i]) == 0:
        df2 = df2.drop(i)

In [25]:
cleaned_text[37]

['foreigners', 'know', 'kuala', 'lumpur', 'city', 'bedbugs']

In [26]:
df2.to_csv('posts1352_preprocessed_check.csv')

In [72]:
df2

,No,Category,SubCategory,Post,HateSpeech,PreprocessedToken
0,1,Race,Chinese,Chinese are pigs!,1,"[chinese, pigs]"
1,2,Race,Indian,I don't like pajeets they smell like curry,1,"[like, pajeets, smell, like, curry]"
2,4,Race,Indian,"Indians always smell stinky and weird, what is...",1,"[indians, smell, stinky, weird]"
3,5,Gender,Female,Women are so emotional they can't handle anyth...,1,"[women, emotional, handle, anything, keep, cry..."
4,6,Race,Malay,Malays are dependant on subsidiaries from the ...,1,"[malays, dependant, subsidiaries, government]"
...,...,...,...,...,...,...
1633,1349,Religion,Buddhism,You cannot build a temple in an area with majo...,1,"[build, temple, area, majority, muslims]"
1634,1350,Religion,Buddhism,"Muslims should not enter temple, or they might...",1,"[muslims, enter, temple, might, accidentaly, c..."
1635,1351,Gender,Male,Men that don't have job are useless piece of s...,1,"[men, job, useless, piece, sheet]"
1636,1352,Gender,Female,Women are dick machines.,1,"[women, dick, machines]"


Making a similar one without removing the None category

In [12]:
df1["Post"] = df1["Post"].apply(lambda x: sent_tokenize(x)) #this only "tokenize" post with multuple lines into sentences

In [13]:
df1 = df1.explode('Post')

df1 = df1.reset_index(drop=True)

In [14]:
#Preprocess the input data
cleaned_text2 = []
counter = 1

for text in df1["Post"]:
    if counter % 100 == 0:
        print(f'Preprocessing row {counter}')
    counter+=1
    cleaned_text2.append(preprocess(text))

Preprocessing row 100
Preprocessing row 200
Preprocessing row 300
Preprocessing row 400
Preprocessing row 500
Preprocessing row 600
Preprocessing row 700
Preprocessing row 800
Preprocessing row 900
Preprocessing row 1000
Preprocessing row 1100
Preprocessing row 1200
Preprocessing row 1300
Preprocessing row 1400
Preprocessing row 1500
Preprocessing row 1600
Preprocessing row 1700
Preprocessing row 1800
Preprocessing row 1900


In [15]:
df1["PreprocessedToken"] = cleaned_text2

In [16]:
df1.to_csv('posts1352_preprocessed_full.csv')

In [17]:
df1

,No,Category,SubCategory,Post,HateSpeech,PreprocessedToken
0,1,Race,Chinese,Chinese are pigs!,1,"[chinese, pig]"
1,2,Race,Indian,I don't like pajeets they smell like curry,1,"[parent, smell, carry]"
2,3,NaN,NaN,Israel is defending itself from Hamas,0,[defend]
3,4,Race,Indian,"Indians always smell stinky and weird, what is...",1,"[always, smell, sticky, weird]"
4,5,Gender,Female,Women are so emotional they can't handle anyth...,1,"[woman, emotional, handle, keep, cry, man, help]"
...,...,...,...,...,...,...
1971,1349,Religion,Buddhism,You cannot build a temple in an area with majo...,1,"[build, muslim]"
1972,1350,Religion,Buddhism,"Muslims should not enter temple, or they might...",1,"[muslim, enter, temple, accidentally, convert,..."
1973,1351,Gender,Male,Men that don't have job are useless piece of s...,1,"[man, job, useless, piece, sheet]"
1974,1352,Gender,Female,Women are dick machines.,1,"[woman, machine]"


In [12]:
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each